# Various Geolocation Analysis

In [ ]:
from bokeh.io import output_notebook,output_file, show, save, curdoc, push_notebook
from bokeh.plotting import figure, output_file, show
from bokeh.models import ColumnDataSource, HoverTool, HBox, VBoxForm, Circle, DataRange1d, \
PanTool, WheelZoomTool, BoxSelectTool,  GMapPlot, GMapOptions
from bokeh.models.widgets import Slider, Select, TextInput, CheckboxButtonGroup
from ipywidgets import interact,widgets
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull
from collections import Counter
import time
from random import randint
import string
import json
import numpy as np
import pandas as pd
from datetime import datetime
from matplotlib.path import Path
from pytz import timezone
from math import floor
from datetime import time as timeclass
from urllib import parse
import requests
from geopy.distance import vincenty
import math
import foliumy
from foliumy.element import IFrame
from IPython.core.display import display, HTML, clear_output
from IPython.display import HTML
import pickle
import os.path

css = """
<style>
.dropdown-menu{
    height: auto;
    max-height: 150px;
    overflow-x: hidden;
}
</style>
"""
HTML(css)
output_notebook()

In [ ]:
def populate_venuetype(x):
    types = ['dining','entertainment','social','sports','corporate','retail']
    result = []
    x = [list(y) for y in set(tuple(y) for y in x)]
    for y in x:
        result.append(y + [types[randint(0,5)]])
    return result

def map_dbscan(latlong_list, ep, mins):
    labels = DBSCAN(eps=ep, min_samples=mins).fit([ [i[0],i[1]] for i in latlong_list] ).labels_
    label_dict = {}
    for i,j in zip(labels, latlong_list):
        if i not in label_dict:
            label_dict[i] = {}
            label_dict[i]['points'] = [ { 'x':float(j[1]), 'y':float(j[0]),'z':1,'type':j[2] } ]
        else:
            label_dict[i]['points'].append( { 'x':float(j[1]), 'y':float(j[0]),'z':1,'type':j[2] } )
    for i in label_dict:
        count = Counter([j['type'] for j in label_dict[i]['points']])
        label_dict[i]['common_type'] = count.most_common()[0][0]
        
        points = np.array( [[j['x'],j['y']] for j in label_dict[i]['points']] )
        if len(points)>2:
            hull = ConvexHull(points)
            label_dict[i]['boundaries'] = {}
            label_dict[i]['boundaries']['simplices'] = points[hull.simplices]
            label_dict[i]['boundaries']['vertices'] = points[hull.vertices]
        
    return label_dict

def get_patches(groupings):
    colors = ["#000000","#008080","#0000FF","#FFFF00","#00FF00","#800080", "#40e0d0","#000FF0","#066FF0","#366FF0"]
    type_to_color = {'dining':0,'entertainment':1,'social':2,'sports':3,'corporate':4,'retail':5}

    group = {}
    for g in groupings:
        group[g] = {}
        group[g]['x'] = []
        group[g]['y'] = []
        group[g]['color'] = []
        for c in groupings[g]:
            if 'boundaries' in groupings[g][c] and c != -1:
                group[g]['x'].append([ j[0] for j in groupings[g][c]['boundaries']['vertices'] ])
                group[g]['y'].append([ j[1] for j in groupings[g][c]['boundaries']['vertices'] ])
                group[g]['color'].append([ colors[type_to_color[groupings[g][c]['common_type']] ] ])
    return group

def get_points(groupings):
    group = {}
    for g in groupings:
        group[g] = {}
        group[g]['x'] = []
        group[g]['y'] = []
        for c in groupings[g]:
            group[g]['x'] = group[g]['x'] + [ j['x'] for j in groupings[g][c]['points'] ]
            group[g]['y'] = group[g]['y'] + [ j['y'] for j in groupings[g][c]['points'] ]
    return group

def checkCoordinate(y, travel_mode, userid):
        result = "User " + travel_mode + ", not in cluster"
        for i in top_users[userid]:
            if i != -1:
                if Path( top_users[userid][i]['boundaries']['vertices'] ).contains_point(y):
                    result = "User " + travel_mode + ", located in cluster [" + str(i) + "], recommending " + \
                    top_users[userid][i]["common_type"] + ' promotions'
        return result
checkin = sc.textFile("geodata/checkins.csv").map(lambda x : x.split(",")).cache()

#### Cell below performs the clustering algorithm (this takes a few seconds)

In [ ]:
topuser_ids = [842, 22, 4985, 4902, 4929, 1201, 578, 111, 1274, 1940, 3480, 5001, 4952]
if os.path.isfile("top_users.pickle"):
    top_users = pickle.load( open( "top_users.pickle", "rb" ) )
else:
    
    checkin_users = checkin.map(lambda x: (x[5], [x[6],x[7]] ) ).groupByKey().\
    mapValues(lambda x : populate_venuetype(x)).cache()
    checkin_users_cluster = checkin_users.mapValues(lambda x : map_dbscan(x, 0.004, 4) ).cache()
    all_users_clusters = checkin_users_cluster.collect()

    #large_clusters = checkin_users_cluster.map(lambda x: (sum( len(x[1][i]['points']) for i in x[1] if i != -1 ), x[0] ) ).\
    #sortByKey(ascending=False).take(20)
    #topuser_ids = [int(c[1]) for c in large_clusters]
    top_users = checkin_users_cluster.map(lambda x: (int(x[0]), x[1]) ).filter(lambda x: x[0] in topuser_ids).collectAsMap()
    pickle.dump( top_users, open("top_users.pickle", "wb") )
users_patches = get_patches(top_users)
users_points = get_points(top_users)

In [ ]:
def display_geolocation_cluster():
    p = figure(title="Click on button \"Simulate\" to begin", title_text_font_size='14pt', x_range=(-74.081,-73.861), y_range=(40.7,40.865), plot_width=740, plot_height=741)
    p.image_url(url=["https://maps.googleapis.com/maps/api/staticmap?center=Manhattan,New+York,NY&zoom=12&size=640x640&maptype=roadmap"],\
                x=-74.081, y=40.865)

    source = ColumnDataSource(
            data=dict(
                x=[0],
                y=[0],
                desc=[ "" ]
            )
        )
    line_source = ColumnDataSource(
            data=dict(
                x=[0],
                y=[0],
            )
        )

    #b = p.line(x=[-74.01, -74.02], y=[40.75, 40.76])
    
    c = p.patches(xs =users_patches[4929]['x'], ys=users_patches[4929]['y'], alpha=0.5, fill_color=users_patches[4929]['color'])
    s = p.square(x= users_points[4929]['x'], y=users_points[4929]['y'], size=2, color="black", alpha=0.3)
    r = p.circle('x', 'y', size=10, color='red', source=source)
    l = p.line('x', 'y', line_width=1, color='red', source=line_source)
    hover = HoverTool(renderers=[r], tooltips=[
                ("(x,y)", "($x, $y)"),
                ("desc", "@desc"),
            ])

    p.add_tools(hover)

    def update_route(lon=-74.01, lat=40.75, lon_line=[-74.01], lat_line=[40.75], travel_mode="None", userid=1):
        r.data_source.data['x'] = [lon]
        r.data_source.data['y'] = [lat]
        l.data_source.data['x'] = lon_line
        l.data_source.data['y'] = lat_line
        r.data_source.data['desc'] = [ checkCoordinate( (lon,lat),travel_mode,userid ) ]
        p.title = checkCoordinate( (lon,lat),travel_mode,userid)
        push_notebook()

    def update_clusters(userid):
        c.data_source.data['xs'] = users_patches[userid]['x']
        c.data_source.data['ys'] = users_patches[userid]['y']
        c.data_source.data['fill_color'] = users_patches[userid]['color']
        s.data_source.data['x'] = users_points[userid]['x']
        s.data_source.data['y'] = users_points[userid]['y']
        push_notebook()

    origin_tb = widgets.Text(description='Origin:', margin=5, value="soho, manhattan")
    desti_tb = widgets.Text(description='Destination:', margin=5, value="5th avenue, manhattan")
    btn = widgets.Button(description='Simulate', margin=5)
    travel_tog = widgets.ToggleButtons(
        description='Travel Modes:',
        options=['driving', 'walking', 'bicycling','transit'],
        margin=5
    )
    user_dd = widgets.Dropdown(
        options={str(i):i for i in topuser_ids},
        value=4929,
        description='Clusters of User:',
        margin=5
    )
    cluster_hbox1 = widgets.HBox(children=[travel_tog,user_dd])
    cluster_hbox2 = widgets.HBox(children=[origin_tb,desti_tb,btn])
    container = widgets.VBox(children=[cluster_hbox1,cluster_hbox2])

    def btn_click(sender):
        def get_routepath(all_steps):
            route_path = []
            def get_mode(temp): 
                if temp['travel_mode'] == 'TRANSIT':
                    return 'IN ' + temp['transit_details']['line']['vehicle']['type'] + ' ' + temp['travel_mode']
                else:
                    return temp['travel_mode']
            for s in all_steps:
                if 'steps' in s:
                    for t in s['steps']:
                        temp = t
                        temp['start_location']['travel_mode'] = get_mode(temp)
                        route_path.append(temp['start_location'])
                    temp = s['steps'][-1]
                    temp['end_location']['travel_mode'] = get_mode(temp)
                    route_path.append(temp['end_location'])
                else:
                    temp = s
                    temp['start_location']['travel_mode'] = get_mode(temp)
                    route_path.append(temp['start_location'])
            temp = all_steps[-1]
            temp['end_location']['travel_mode'] = get_mode(temp)
            route_path.append(temp['end_location'])
            return route_path

        args = {"origin": origin_tb.value, "destination": desti_tb.value, "mode": travel_tog.value}
        url = "https://maps.googleapis.com/maps/api/directions/json?{}".format(parse.urlencode(args))
        route = requests.get(url).json()
        all_steps = route['routes'][0]['legs'][0]['steps']
        route_path = get_routepath(all_steps)
        new_route = []
        for idx, r in enumerate(route_path):
            if idx == 0:
                new_route.append(r)
            else:
                distance = vincenty((r['lat'],r['lng']), (route_path[idx-1]['lat'],route_path[idx-1]['lng'])).miles
                division = 1
                max_dist = 0.15
                if distance > max_dist:
                    division = math.ceil(distance/max_dist)
                    lat_diff = (r['lat'] - route_path[idx-1]['lat'])/division
                    lng_diff = (r['lng'] - route_path[idx-1]['lng'])/division
                    for i in range(1,division+1):
                        new_coord = {}
                        new_coord['lat'] = route_path[idx-1]['lat'] + i*lat_diff
                        new_coord['lng'] = route_path[idx-1]['lng'] + i*lng_diff
                        new_coord['travel_mode'] = route_path[idx-1]['travel_mode']
                        new_route.append(new_coord)
                else:
                    new_route.append(r)
        route_line = {}
        route_line['lng'] = []
        route_line['lat'] = []
        for r in new_route:
            route_line['lng'].append(r['lng'])
            route_line['lat'].append(r['lat'])
            update_route(lon=r['lng'], lat=r['lat'], lon_line=route_line['lng'], lat_line=route_line['lat'],
                         travel_mode=r['travel_mode'],userid=user_dd.value)
            time.sleep(0.3)
        return print("Simulation Completed")

    def dd_change(sender):
        update_clusters(user_dd.value)

    btn.on_click(btn_click)
    user_dd.on_trait_change(dd_change)
    display(container, show(p))

# Geolocation Clustering (social app data)
### Simulation of user's movement,  in and out of cluster when travelling various routes
### Recommending users different type of promotion when they are in each cluster
<font color='red'>*rerun the cell with CTRL+Enter if the Simulate button is not working</font> 

In [ ]:
display_geolocation_cluster()

#### Cell below performs Data Transformation (this takes a few seconds)

In [ ]:
datestr = '%Y-%m-%dT%H:%M:%SZ'
def to_eastern(x):
    return datetime.strptime(x,datestr).replace(tzinfo=timezone('utc')).astimezone(tz=timezone('US/Eastern'))
def get_weekendday(x):
    return 'Weekend' if x.isoweekday() in [6,7] else 'Weekday'

if os.path.isfile("checkin_hours.pickle"):
    checkin_hours = pickle.load( open( "checkin_hours.pickle", "rb" ) )
else:
    venue = sqlContext.read.format('com.databricks.spark.csv').options(inferschema='true').load('geodata/venues.csv').rdd
    checkin_names = checkin.map(lambda x: (int(x[8]),x)).leftOuterJoin(venue.map(lambda x: (int(x[0]),x[1]) ) ).map(lambda x: x[1][0]+[x[1][1]] )
    checkin_hours = checkin_names.map(lambda x: ( 
         ( get_weekendday(to_eastern(x[4])),  round(to_eastern(x[4]).hour + to_eastern(x[4]).minute/60,1) ),  [x[6],x[7],x[9]] )  ).\
    groupByKey().mapValues(lambda x: {'y':[i[0] for i in x], 'x':[i[1] for i in x], 'name':[i[2] if i[2] else "-" for i in x ]}).collectAsMap()
    pickle.dump( checkin_hours, open("checkin_hours.pickle", "wb") )

In [ ]:
def geolocation_checkin_timelapse():
    
    hour_p = figure(title="Click on button \"Timelapse\" to begin", title_text_font_size='14pt', x_range=(-74.081,-73.861), 
                    y_range=(40.7,40.865), plot_width=740, plot_height=741)
    hour_p.image_url(
        url=["https://maps.googleapis.com/maps/api/staticmap?center=Manhattan,New+York,NY&zoom=12&size=640x640&maptype=roadmap"],\
                x=-74.081, y=40.865)


    hour_source = ColumnDataSource(
            data=dict(
                x=checkin_hours[('Weekend',0)]['x']+checkin_hours[('Weekday',0)]['x'],
                y=checkin_hours[('Weekend',0)]['y']+checkin_hours[('Weekday',0)]['y'],
                desc=checkin_hours[('Weekend',0)]['name']+checkin_hours[('Weekday',0)]['name']
            )
        )
    
    hour_c = hour_p.circle('x', 'y', alpha=0.5, color='red', source=hour_source)

    hour_hover = HoverTool(renderers=[hour_c], tooltips=[
                ("(x,y)", "($x, $y)"),
                ("desc", "@desc"),
            ])
    hour_p.add_tools(hour_hover)
    
    def decimalhour_to_hour(time_hours):
        time_minutes = time_hours * 60
        time_seconds = time_minutes * 60

        hours_part   = floor(time_hours)
        minutes_part = floor(time_minutes % 60)
        seconds_part = floor(time_seconds % 60)
        return timeclass(hours_part, minutes_part, seconds_part).strftime('%H:%M%P')

    def update_hour(hour=0,day="Whole week"):
        xlist,ylist,namelist = [],[],[]
        if day=='Weekend':
            xlist = checkin_hours[('Weekend',hour)]['x']
            ylist = checkin_hours[('Weekend',hour)]['y']
            namelist = checkin_hours[('Weekend',hour)]['name']
        elif day=='Weekday':
            xlist = checkin_hours[('Weekday',hour)]['x']
            ylist = checkin_hours[('Weekday',hour)]['y']
            namelist = checkin_hours[('Weekday',hour)]['name']
        else:
            xlist = checkin_hours[('Weekend',hour)]['x'] + checkin_hours[('Weekday',hour)]['x']
            ylist = checkin_hours[('Weekend',hour)]['y'] + checkin_hours[('Weekday',hour)]['y']
            namelist = checkin_hours[('Weekend',hour)]['name'] + checkin_hours[('Weekday',hour)]['name']

        hour_c.data_source.data['x'] = xlist
        hour_c.data_source.data['y'] = ylist
        hour_c.data_source.data['desc'] = namelist
        hour_p.title = "User activity during the " + day.lower() + " at " + str(decimalhour_to_hour(hour))
        push_notebook()

    hour_slider = widgets.FloatSlider(
        value=0,
        min=0,
        max=23.9,
        step=0.1,
        description='Hour of the day: ',
        margin=5

    )
    hour_btn = widgets.Button(description='Timelapse', margin=5)

    def hour_change(sender):
        update_hour(hour_slider.value, day_select.value)
    def hour_btn_click(sender):
        for i in range(0,239,1):
            update_hour(i/10)
            hour_slider.value = i/10
            time.sleep(0.01)

    hour_slider.on_trait_change(hour_change)
    hour_btn.on_click(hour_btn_click)
    day_select = widgets.ToggleButtons(
        description="Day of the week:",
        options=['Weekend','Weekday','Whole week'],
        value='Whole week',
        margin=5
    )

    day_select.on_trait_change(hour_change)

    hbox1 = widgets.HBox(children=[hour_slider,hour_btn])
    hbox2 = widgets.HBox(children=[day_select,hbox1])
    display(hbox2, show(hour_p))

# Analysis on the change in user check-ins (social app data)
### Identify where are the key user activities during different time periods
<font color='red'>*rerun the cell with CTRL+Enter if the TimeLapse button is not working</font> 

In [ ]:
geolocation_checkin_timelapse()

#### Cell below performs Data Loading and Transformation (this may take a few minutes)

In [ ]:
SF_COORDINATES = (37.76, -122.45)  
if os.path.isfile("SF_crime.pickle"):
    SF_crime = pickle.load( open( "SF_crime.pickle", "rb" ) )
else:
    SF_crime = pd.read_csv('geodata/SFPD_Incidents_2015.csv', parse_dates=['Date']).sample(frac=0.01)
    pickle.dump( SF_crime, open("SF_crime.pickle", "wb") )


mc = foliumy.MarkerCluster()

i = 0
#add a marker for every record in the filtered data, use a clustered view
for each in SF_crime.iterrows():

    incident_num = each[1]['IncidntNum']
    crime_cat = each[1]['Category']
    crime_desc = each[1]['Descript']
    crime_date = each[1]['Date']
    crime_time = each[1]['Time']
    crime_res = each[1]['Resolution']

    bubble_details = IFrame(html="<html>Incident No: {}<br>Category: {}<br>Description: {}<br>Date: {}<br>\
    Time: {}<br>Resolution: {}</html>".format(incident_num, crime_cat, crime_desc, crime_date, crime_time, crime_res),\
                                           width="300px",height="150px")
    marker = foliumy.Marker(location=[each[1]['Y'],each[1]['X']],
                           popup = foliumy.Popup(html=bubble_details))
    mc.add_child(marker)
    i+=1
#create empty map zoomed in on San Francisco
sf_map = foliumy.Map(location=SF_COORDINATES, zoom_start=12)

# Crime distribution in San Francisco (crime data)
### See the distribution of crime in different region of the city
#### *Data sampled down to 1% of the actual figures

In [ ]:
sf_map.add_child(mc)

In [ ]:
pd.unique(SF_crime.Category)

# create a new column in SF_crime and initialize to 0
SF_crime['PeriodOfDay'] = 0

SF_week_days = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
SF_week_ends = ['Saturday', 'Sunday']

SF_crime.loc[SF_crime['DayOfWeek'].isin(SF_week_days), 'PeriodOfDay'] = 'WEEKDAYS'
SF_crime.loc[SF_crime['DayOfWeek'].isin(SF_week_ends), 'PeriodOfDay'] = 'WEEKENDS'

In [ ]:
def generate_spec_map(x,y):
    try:
        #definition of the boundaries in the map
        district_sf = r'geodata/sfpddistricts.geojson'
        
        if os.path.isfile("SF_district_df.pickle"):
            SF_district_df = pickle.load( open( "SF_district_df.pickle", "rb" ) )
        else:
            if x == 'ALLDAYS':
                SF_crime_df = SF_crime.loc[(SF_crime.Category == str(y))]
            else:
                SF_crime_df = SF_crime.loc[(SF_crime.PeriodOfDay == str(x)) & (SF_crime.Category == str(y))]

            #unique districts and number of crimes initialized to 0
            SF_districts = pd.unique(SF_crime.PdDistrict)
            SF_districts = pd.DataFrame(SF_districts)
            SF_districts.columns = ['District']
            SF_districts['Number'] = 0

            #calculating total number of incidents per district
            SF_crime_df = pd.DataFrame(SF_crime_df['PdDistrict'].value_counts().astype(float))  
            SF_crime_df.to_json('SF_crime_spec_agg.json')  
            SF_crime_df = SF_crime_df.reset_index()  
            SF_crime_df.columns = ['District', 'Number']

            SF_district_df = SF_districts.merge(SF_crime_df, how="left", left_on="District", \
                                                right_on="District")[['District','Number_y']].fillna(value=0)
            SF_district_df.columns = ['District', 'Number']
            pickle.dump( SF_district_df, open("SF_district_df.pickle", "wb") )
        
        
             
        #creation of the choropleth
        map_spec = foliumy.Map(location=SF_COORDINATES, zoom_start=12)  

        map_spec.choropleth(geo_path = district_sf, 
                            data_out = 'SF_crime_spec_agg.json', 
                            data = SF_district_df,
                            columns = ['District', 'Number'],
                            key_on = 'feature.properties.DISTRICT',
                            threshold_scale=[10,100,200,300,400,500],
                            fill_color = 'YlOrRd', 
                            fill_opacity = 0.7, 
                            line_opacity = 0.2,
                            legend_name = 'Number of incidents per district')
        return map_spec
    
    except:
        raise

def generate_choropleth():
    widgets.interact(generate_spec_map, x=('WEEKDAYS','WEEKENDS','ALLDAYS'), 
                     y=('NON-CRIMINAL', 'ASSAULT', 'OTHER OFFENSES', 'VANDALISM',
           'DRUNKENNESS', 'WARRANTS', 'MISSING PERSON', 'DRUG/NARCOTIC',
           'ROBBERY', 'LARCENY/THEFT', 'BURGLARY', 'SECONDARY CODES',
           'RUNAWAY', 'VEHICLE THEFT', 'SEX OFFENSES, FORCIBLE',
           'SUSPICIOUS OCC', 'WEAPON LAWS', 'SUICIDE', 'FRAUD', 'LIQUOR LAWS',
           'TRESPASS', 'FORGERY/COUNTERFEITING', 'STOLEN PROPERTY', 'BRIBERY',
           'EXTORTION', 'ARSON', 'DRIVING UNDER THE INFLUENCE', 'PROSTITUTION',
           'DISORDERLY CONDUCT', 'EMBEZZLEMENT', 'KIDNAPPING',
           'FAMILY OFFENSES', 'BAD CHECKS', 'LOITERING', 'GAMBLING',
           'SEX OFFENSES, NON FORCIBLE', 'PORNOGRAPHY/OBSCENE MAT', 'TREA'))

# Identify Criminal Hotspots
### Analysis based on time period and crime type
#### *Data sampled down to 1% of the actual figures

In [ ]:
generate_choropleth()